<a href="https://colab.research.google.com/github/seodalzzz/ESAA/blob/Mini-Project/summer_project_Clustering_for_User_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
###구글드라이브 마운트 & 기본 라이브러리 로드


In [290]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [291]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')



---
###사용자 입력 데이터 Standardization


In [292]:
DF_2022_preknn=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/summer_project/dataset/data_from2022.csv")

In [293]:
col = ['MESURE_AGE_CO','SEXDSTN_FLAG_CD','MESURE_IEM_001_VALUE','MESURE_IEM_002_VALUE','MESURE_IEM_003_VALUE','MESURE_IEM_005_VALUE','MESURE_IEM_006_VALUE','MESURE_IEM_012_VALUE','MESURE_IEM_018_VALUE']
col_std=['MESURE_AGE_CO','MESURE_IEM_001_VALUE','MESURE_IEM_002_VALUE','MESURE_IEM_003_VALUE','MESURE_IEM_005_VALUE','MESURE_IEM_006_VALUE','MESURE_IEM_012_VALUE','MESURE_IEM_018_VALUE']
DF_2022_preknn=DF_2022_preknn[col]
DF_2022_std = DF_2022_preknn[col_std]

In [294]:
DF_2022_preknn.head(5)

,MESURE_AGE_CO,SEXDSTN_FLAG_CD,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE
0,15.0,M,175.2,89.0,14.3,66.0,125.0,18.9,29.0
1,63.0,M,173.0,72.2,23.8,77.0,121.0,15.3,24.1
2,23.0,M,179.1,72.7,14.8,97.0,148.0,14.3,22.7
3,63.0,M,166.2,74.8,23.6,81.0,136.0,4.0,27.1
4,69.0,F,154.4,57.6,36.6,62.0,121.0,15.5,24.2


In [295]:
DF_2022_std.head(5)

,MESURE_AGE_CO,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE
0,15.0,175.2,89.0,14.3,66.0,125.0,18.9,29.0
1,63.0,173.0,72.2,23.8,77.0,121.0,15.3,24.1
2,23.0,179.1,72.7,14.8,97.0,148.0,14.3,22.7
3,63.0,166.2,74.8,23.6,81.0,136.0,4.0,27.1
4,69.0,154.4,57.6,36.6,62.0,121.0,15.5,24.2


In [296]:
mean_list=range(DF_2022_std.shape[1])
sd_list=range(DF_2022_std.shape[1])

mean_list=pd.DataFrame(mean_list)
sd_list=pd.DataFrame(sd_list)

In [297]:
for i in range(DF_2022_std.shape[1]):
  mean_list.iloc[i]=np.mean(DF_2022_std[col_std[i]])
  sd_list.iloc[i]=np.std(DF_2022_std[col_std[i]])

In [298]:
print(mean_list)
print(sd_list)

            0
0   25.208031
1  165.068139
2   63.975403
3   26.227990
4   75.532216
5  124.932728
6   10.111001
7   23.336163
           0
0  18.411967
1  10.136470
2  15.601813
3  11.450375
4  21.196510
5  61.551252
6  10.148138
7  12.433217


In [299]:
user_input=pd.DataFrame({'MESURE_AGE_CO':[58],
                         'SEXDSTN_FLAG_CD':['F'],
                         'MESURE_IEM_001_VALUE':[156.5],
                         'MESURE_IEM_002_VALUE':[55.5],
                         'MESURE_IEM_003_VALUE':[29.6],
                         'MESURE_IEM_005_VALUE':[78],
                         'MESURE_IEM_006_VALUE':[125],
                         'MESURE_IEM_012_VALUE':[21.6],
                         'MESURE_IEM_018_VALUE':[22.7]})
user_input

,MESURE_AGE_CO,SEXDSTN_FLAG_CD,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE
0,58,F,156.5,55.5,29.6,78,125,21.6,22.7


In [300]:
# 성별 제외 데이터프레임 입력받아 각 변수별 평균, 분산으로 standardization
def standardize_userData(data):
  data_stand=data
  for i in range(data.shape[1]):
    data_stand[col_std[i]]=((data[col_std[i]]-mean_list.iloc[i])/sd_list.iloc[i])
  return data_stand

In [301]:
# 성별 포함 데이터프레임 input을 standardization
def standardization(user_input):
  user_data_for_std=user_input[col_std]
  user_sex=user_input['SEXDSTN_FLAG_CD']
  user_data_stand=standardize_userData(user_data_for_std)
  user_data_stand.insert(loc=1,column='SEXDSTN_FLAG_CD',value=user_sex)
  return user_data_stand

In [302]:
user_data_stand=standardization(user_input)
user_data_stand

,MESURE_AGE_CO,SEXDSTN_FLAG_CD,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE
0,1.781014,F,-0.845278,-0.543232,0.294489,0.116424,0.001093,1.132129,-0.051166




---
###K-Means (군집 별 중심 구하기)


In [303]:
from sklearn.cluster import KMeans

In [304]:
DF_2022=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/summer_project/dataset/DF_2022_KNN.csv")

In [305]:
# 공통데이터
DF_2022 = DF_2022[col]
DF_2022 = pd.get_dummies(data=DF_2022)

DF_kmeans = DF_2022.copy()
DF_meanshift = DF_2022.copy()
DF_gmm = DF_2022.copy()
DF_dbscan = DF_2022.copy()

In [306]:
kmeans = KMeans(n_clusters=7, init='k-means++', max_iter=300,random_state=42)
kmeans.fit(DF_2022)
print(kmeans.labels_)

[0 4 0 ... 0 0 2]


In [307]:
kmeans.cluster_centers_

array([[-3.28914541e-01,  7.49721521e-01,  8.26464411e-02,
        -7.77188312e-01, -5.56592459e-02, -2.75959573e-02,
         1.33990851e-02, -1.18174859e-01,  7.13924690e-03,
         9.92860753e-01],
       [-6.45059726e-01, -1.17302187e+00, -1.09937348e+00,
         6.57951378e-02, -1.24658359e-01, -1.30461019e-01,
        -6.44496749e-01, -2.76862035e-01,  5.54837027e-01,
         4.45162973e-01],
       [-4.06785496e-01, -4.40959913e-01, -3.94045558e-01,
         4.79197138e-01, -1.13934301e-01, -1.12828442e-01,
         7.64668333e-01, -7.42774520e-02,  9.64626528e-01,
         3.53734718e-02],
       [-1.90641030e-01,  8.75325240e-01,  1.49232300e+00,
         4.17177668e-01,  1.75524528e-01,  7.69116713e-02,
        -5.01071496e-01,  4.53109745e-01,  8.26378156e-02,
         9.17362184e-01],
       [ 2.12037156e+00, -3.84170073e-01, -1.01424625e-02,
         2.92892688e-01,  1.91928547e-01,  2.20197268e-01,
         9.68139312e-02,  9.51088036e-02,  4.85110031e-01,
         5.

In [308]:
print(user_input.columns)
print(DF_2022.columns)

Index(['MESURE_AGE_CO', 'SEXDSTN_FLAG_CD', 'MESURE_IEM_001_VALUE',
       'MESURE_IEM_002_VALUE', 'MESURE_IEM_003_VALUE', 'MESURE_IEM_005_VALUE',
       'MESURE_IEM_006_VALUE', 'MESURE_IEM_012_VALUE', 'MESURE_IEM_018_VALUE'],
      dtype='object')
Index(['MESURE_AGE_CO', 'MESURE_IEM_001_VALUE', 'MESURE_IEM_002_VALUE',
       'MESURE_IEM_003_VALUE', 'MESURE_IEM_005_VALUE', 'MESURE_IEM_006_VALUE',
       'MESURE_IEM_012_VALUE', 'MESURE_IEM_018_VALUE', 'SEXDSTN_FLAG_CD_F',
       'SEXDSTN_FLAG_CD_M'],
      dtype='object')


In [309]:
#user_input getdummies 처럼 만들어주는 함수
def user_input_likeGetDummies(user_input_std):
  if user_input_std['SEXDSTN_FLAG_CD'][0]=='F':
    user_input_std['SEXDSTN_FLAG_CD_F']=1
    user_input_std['SEXDSTN_FLAG_CD_M']=0
  else:
    user_input_std['SEXDSTN_FLAG_CD_F']=0
    user_input_std['SEXDSTN_FLAG_CD_M']=1
  user_input_std=user_input_std.drop(columns='SEXDSTN_FLAG_CD',axis=1)
  return user_input_std

In [310]:
user_data_stand=user_input_likeGetDummies(user_data_stand)
user_data_stand

,MESURE_AGE_CO,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE,SEXDSTN_FLAG_CD_F,SEXDSTN_FLAG_CD_M
0,1.781014,-0.845278,-0.543232,0.294489,0.116424,0.001093,1.132129,-0.051166,1,0


In [311]:
user_data_stand['SEXDSTN_FLAG_CD_F']=(user_data_stand['SEXDSTN_FLAG_CD_F']-np.mean(DF_2022['SEXDSTN_FLAG_CD_F']))/np.std(DF_2022['SEXDSTN_FLAG_CD_F'])
user_data_stand['SEXDSTN_FLAG_CD_M']=(user_data_stand['SEXDSTN_FLAG_CD_M']-np.mean(DF_2022['SEXDSTN_FLAG_CD_M']))/np.std(DF_2022['SEXDSTN_FLAG_CD_M'])

In [312]:
from scipy.spatial import distance

In [313]:
# 중심 좌표가 user data의 거리가 가장 가까운 군집을 구하는 함수
def min_dist_index(user_data_stand,centers):
  distance_list=np.zeros(7)
  for i in range(7):
    distance_list[i]=distance.euclidean(user_data_stand.iloc[0],kmeans.cluster_centers_[i])
  return np.argmin(distance_list)

In [314]:
closest_cluster=min_dist_index(user_data_stand,kmeans.cluster_centers_)
closest_cluster

4



---
###최종 실행 코드


In [315]:
user_input=pd.DataFrame({'MESURE_AGE_CO':[20],
                         'SEXDSTN_FLAG_CD':['M'],
                         'MESURE_IEM_001_VALUE':[168.1],
                         'MESURE_IEM_002_VALUE':[75.4],
                         'MESURE_IEM_003_VALUE':[24],
                         'MESURE_IEM_005_VALUE':[86],
                         'MESURE_IEM_006_VALUE':[146],
                         'MESURE_IEM_012_VALUE':[10.1],
                         'MESURE_IEM_018_VALUE':[26.7]})
user_input

,MESURE_AGE_CO,SEXDSTN_FLAG_CD,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE
0,20,M,168.1,75.4,24,86,146,10.1,26.7


In [316]:
user_data_stand=standardization(user_input)
user_data_stand

,MESURE_AGE_CO,SEXDSTN_FLAG_CD,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE
0,-0.282861,M,0.299104,0.732261,-0.194578,0.493845,0.342272,-0.001084,0.270552


In [317]:
user_data_stand=user_input_likeGetDummies(user_data_stand)
user_data_stand

,MESURE_AGE_CO,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE,SEXDSTN_FLAG_CD_F,SEXDSTN_FLAG_CD_M
0,-0.282861,0.299104,0.732261,-0.194578,0.493845,0.342272,-0.001084,0.270552,0,1


In [318]:
user_data_stand['SEXDSTN_FLAG_CD_F']=(user_data_stand['SEXDSTN_FLAG_CD_F']-np.mean(DF_2022['SEXDSTN_FLAG_CD_F']))/np.std(DF_2022['SEXDSTN_FLAG_CD_F'])
user_data_stand['SEXDSTN_FLAG_CD_M']=(user_data_stand['SEXDSTN_FLAG_CD_M']-np.mean(DF_2022['SEXDSTN_FLAG_CD_M']))/np.std(DF_2022['SEXDSTN_FLAG_CD_M'])
user_data_stand

,MESURE_AGE_CO,MESURE_IEM_001_VALUE,MESURE_IEM_002_VALUE,MESURE_IEM_003_VALUE,MESURE_IEM_005_VALUE,MESURE_IEM_006_VALUE,MESURE_IEM_012_VALUE,MESURE_IEM_018_VALUE,SEXDSTN_FLAG_CD_F,SEXDSTN_FLAG_CD_M
0,-0.282861,0.299104,0.732261,-0.194578,0.493845,0.342272,-0.001084,0.270552,-0.825423,0.825423


In [319]:
closest_cluster=min_dist_index(user_data_stand,kmeans.cluster_centers_)
closest_cluster

0

In [320]:
distance_list

array([  1.96174922,   1.94663296,   1.11140284,   2.38741408,
         2.15599915, 215.88006766, 176.66091219])



---
### 이야기 해봐야 할 것
성별 더미변수 표준화
